# 需求

input里的数据源自财务指标计算模板
我现在是想实现直接从财务报表中抓取数据，计算出input中的这些数，填入其中

## 准备部分

### Import

In [1]:
import numpy as np
import pandas as pd

### def

#### 基础公式

In [2]:
# excel里列的字母转数字
def col_to_num(col_str):
    num = 0
    for i, c in enumerate(reversed(col_str)):
        num += (ord(c) - ord('A') + 1) * (26 ** i)
    return num - 1

# excel里行的数字转换
def row_to_num(num):
    return int(num) - 2

#### 计算公式

In [62]:
def EBITDA(营业利润, 财务费用, 折旧费, 公允价值变动, 投资收益, 取得投资收益收到的现金, 政府补助, 经营租赁费用调整, 资本化开发成本, 勘探费用):
    return 营业利润 + 财务费用 + 折旧费 - 公允价值变动 - 投资收益 + 取得投资收益收到的现金 + 政府补助 + 经营租赁费用调整 - 资本化开发成本 + 勘探费用

def EBIT(营业利润, 财务费用, 利息收入, 公允价值变动, 投资收益, 对联营企业和合营企业的投资收益, 政府补助, 经营租赁的利息调整):
    return 营业利润 + 财务费用 + 利息收入 - 公允价值变动 - 投资收益 + 对联营企业和合营企业的投资收益 + 政府补助 + 经营租赁的利息调整

def FOCF(经营活动产生的现金流量净额, 购建固定资产无形资产和其他长期资产支付的现金, 取得投资收益收到的现金, 收到其他与投资活动有关的现金附注利息部分, 分配股利利润或偿付利息支付的现金, 对所有者或股东的分配, 经营租赁折旧调整, 资本化开发成本):
    return 经营活动产生的现金流量净额 - 购建固定资产无形资产和其他长期资产支付的现金 + 取得投资收益收到的现金 + 收到其他与投资活动有关的现金附注利息部分 - 分配股利利润或偿付利息支付的现金 - 对所有者或股东的分配 + 经营租赁折旧调整 - 资本化开发成本

def FFO(EBITDA, 利息费用, 利息收入, 所得税费用, 经营租赁费用调整, 经营租赁折旧调整, 资本化利息):
    return EBITDA - 利息费用 + 利息收入 - 所得税费用 - 经营租赁费用调整 + 经营租赁折旧调整 - 资本化利息

def Total_debt(短期借款, 应付利息, 一年内到期的长期借款, 一年内到期的应付债券, 其它流动负债短期应付债券, 一年内应付融资租赁款, 长期借款, 应付债券, 长期应付融资租赁款, 重大合同及履行状况担保情况, 货币资金, 以公允价值计量且其变动计入当期损益的金融资产, 其他货币资金, 卖出回购金融资产款, 特定行业或公司现金盈余不做调整扣除的部分加回, 经营租赁调整, 永续债):
    return 短期借款 + 应付利息 + 一年内到期的长期借款 + 一年内到期的应付债券 + 其它流动负债短期应付债券 + 一年内应付融资租赁款 + 长期借款 + 应付债券 + 长期应付融资租赁款 + 重大合同及履行状况担保情况 - (货币资金 + 以公允价值计量且其变动计入当期损益的金融资产 - 其他货币资金) * 0.75 + 卖出回购金融资产款 + 特定行业或公司现金盈余不做调整扣除的部分加回 + 经营租赁调整 + 永续债

def Capital(所有者权益合计, 短期借款, 应付利息, 一年内到期的长期借款, 一年内到期的应付债券, 其它流动负债短期应付债券, 一年内应付融资租赁款, 长期借款, 应付债券, 长期应付融资租赁款, 递延所得税负债, 重大合同及履行状况担保情况, 货币资金, 以公允价值计量且其变动计入当期损益的金融资产, 其他货币资金, 卖出回购金融资产款, 特定行业或公司现金盈余不做调整扣除的部分加回, 经营租赁调整, 永续债):
    return 所有者权益合计 + 短期借款 + 应付利息 + 一年内到期的长期借款 + 一年内到期的应付债券 + 其它流动负债短期应付债券 + 一年内应付融资租赁款 + 长期借款 + 应付债券 + 长期应付融资租赁款 + 递延所得税负债 + 重大合同及履行状况担保情况 - (货币资金 + 以公允价值计量且其变动计入当期损益的金融资产 - 其他货币资金) * 0.75 + 卖出回购金融资产款 + 特定行业或公司现金盈余不做调整扣除的部分加回 + 经营租赁调整 + 永续债

def EBITDA_profit_rate(EBITDA, 营业收入):
    return EBITDA / 营业收入

def Capital_RR(EBIT, 资本):
    return EBIT / 资本

def Operating_cash_to_debt(FFO, 总负债):
    return FFO / 总负债

def debt_to_PBITA(总负债, EBITDA):
    return 总负债 / EBITDA

def FOCF_to_debt(FOCF, 总负债):
    return FOCF / 总负债

def EBITDA_to_interest_expense(EBITDA, 财务费用, 资本化利息, 经营租赁的利息调整):
    return EBITDA / (财务费用 + 资本化利息 + 经营租赁的利息调整)

### 读取数据

In [31]:
# 设置路径
report_path = "../评级-芒市新能源 - 副本.xlsx"
report_2022_path = "../data/2022年年报.xlsx"
report_2023_path = "../data/2023年年报.xlsx"
company_info_path = "../data/大唐芒市新能源开发有限公司 2024年5月.xlsx"
revenue_path = "../data/副本芒市新能源2024年收入预测.xlsx"
back_data_path = "../数据底稿.xlsx"

# 读取文件以及需要的数据
report = pd.read_excel(report_path, sheet_name="Inputs")
report_2022 = pd.read_excel(report_2022_path, sheet_name=None)
report_2023 = pd.read_excel(report_2023_path, sheet_name=None)
company_info = pd.read_excel(company_info_path, sheet_name=None)
revenue = pd.read_excel(revenue_path)
back_data = pd.read_excel(back_data_path)

f:\Tools\miniconda\envs\FinaCoding\Lib\site-packages\openpyxl\worksheet\_read_only.py:85: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():
f:\Tools\miniconda\envs\FinaCoding\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


## 数据预处理

In [32]:
# 2022年年报数据
bal_sheet_2022 = report_2022["NB003-资产负债表"]
# # 设置第四行为列名
# bal_sheet_2022.columns = bal_sheet_2022.iloc[2]
# # 删除前三行以及最后两行
# bal_sheet_2022 = bal_sheet_2022.drop([0, 1, 2])
# bal_sheet_2022 = bal_sheet_2022.drop(bal_sheet_2022.tail(2).index)
# # 删除第四列之后的列
# bal_sheet_2022 = bal_sheet_2022.iloc[:, :col_to_num("D") + 1]

# # 展示数据
# bal_sheet_2022

bal_sheet_2022_con = report_2022["NB004-资产负债表（续）"]
profit_sheet_2022 = report_2022["NB005-利润表"]
cash_sheet_2022 = report_2022["NB006-现金流量表"]
ecoIndex_sheet_2022 = report_2022["NB014-技术经济指标表"]
cost_sheet_2022 = report_2022["NB020-成本费用情况表"]

# 2023年年报数据
bal_sheet_2023 = report_2023["Z01 资产负债表"]
profit_sheet_2023 = report_2023["Z02 利润表"]
cash_sheet_2023 = report_2023["Z03 现金流量表"]
cost_sheet_2023 = report_2023["Z20 成本费用情况表"]
ecoIndex_sheet_2023 = report_2023["DT67 技术经济指标表"]

# 公司信息
fmdm = company_info["FMDM 封面代码"]
consSheet = company_info["CWYB2401 大唐集团参数表"]
comp_bal_sheet = company_info["CWYB2402 资产负债表"]
comp_profit_sheet = company_info["CWYB2403 利润表"]
comp_cash_sheet = company_info["CWYB2404 现金流量表"]
comp_profit_detial = company_info["CWYB2406 利润明细表"]
comp_profit_detial_con = company_info["CWYB2407 利润明细表（续）"]
comp_cash_info = company_info["CWYB2405 应交款项情况表"]
comp_product_cost = company_info["CWYB2408 电热产品成本表"]
tech_econ_indicators = company_info["CWYB2410 技术经济指标表"]
cost_expenses = company_info["CWYB2409 成本费用情况表"]
other_product_costs = company_info["CWYB2411 电热产品其他费用明细表"]
labor_costs = company_info["CWYB2412 人工成本表"]
state_control_stats = company_info["JGBS2405 中央企业“两金”压控工作统计表（国资委）"]
capital_budget_completion = company_info["CWYB2415 资本性预算完成情况表"]
capital_budget_detail = company_info["CWYB2416 资本性预算完成情况明细表"]
economic_value_added = company_info["CWYB2417 经济增加值明细表"]
new_unit_indicators = company_info["GLYB2401 新投机组主要指标表"]
renewable_energy_revenue = company_info["GLYB2402 可再生能源电费回收情况表"]
group_financing_plan = company_info["GLYB2403 集团公司融资计划情况表"]
tax_policy_execution = company_info["GLYB2406 集团公司税收政策执行情况表"]
funds_management = company_info["GLYB2407 资金集中管理情况表"]
coal_production_cost = company_info["GLYB2409 煤炭生产成本费用情况表"]
coal_production_sales = company_info["GLYB2410 煤炭生产销售情况表"]
construction_projects = company_info["GLYB2411 在建工程"]
coal_operational_indicators = company_info["GLYB2416 煤炭企业主要经营指标"]
account_details = company_info["GLYB2417 账户信息明细表"]
electricity_sales_revenue = company_info["GLYB2419 电力销售收入明细表"]
fuel_costs = company_info["GLYB2420 发电供热耗用燃料成本表"]
fuel_purchase_monthly = company_info["GLYB2421 燃料采购费用情况表（本月）"]
fuel_purchase_yearly = company_info["GLYB2422 燃料采购费用情况表（本年）"]
marginal_profit = company_info["GLYB2425 燃机边际利润统计表"]
safety_production_feb = company_info["LSTRRRMH 安全生产费情况表_2月及以后"]
safety_production_detail = company_info["GLYB2427 安全生产费使用明细表"]
external_donations = company_info["KB23JZJB 对外捐赠支出月表"]
internal_transactions_balance = company_info["NB2402 内部交易余额统计"]
internal_sales_services = company_info["NB2403 内部销售商品、提供劳务情况表"]
internal_cash_flow = company_info["NB2401 内部现金流量表"]
financial_report_sasac = company_info["JGBS2401 企业财务快报（国资委）"]
financial_report_mof = company_info["JGBS2402 企业财务快报（财政部）"]
debt_risk_monitoring = company_info["KB23JWJCX 中央企业债务风险监测表"]
bond_issuance = company_info["KB23ZQFXX 中央企业债券发行情况表"]
bond_financing_plan = company_info["KB23ZQRZX 中央企业债券融资计划表"]
additional_reports = company_info["KB23FBWD 附报文档"]
hidden_sheet = company_info["HIDDENSHEETNAME"]

# 收入预测
revenue = revenue

# 数据底稿
back_data = back_data


## 计算部分

### 数据分配

In [63]:
# 建立字典
raw_data_2022 = {"EBITDA": 0,
                 "EBIT": 0,
                 "自由运营现金流(FOCF)": 0,
                 "经营活动产生的现金(FFO)": 0,
                 "总负债": 0,
                 "资本": 0,
                 "EBITDA利润率": 0,
                 "资本回报率": 0,
                 "经营活动产生的资金/债务": 0,
                 "债务/息税摊折前利润": 0,
                 "自由运营现金流/债务": 0,
                 "息税摊折前利润 / 利息支出": 0
                }

raw_data_2022 = {"EBITDA": 0,
                 "EBIT": 0,
                 "自由运营现金流(FOCF)": 0,
                 "经营活动产生的现金(FFO)": 0,
                 "总负债": 0,
                 "资本": 0,
                 "EBITDA利润率": 0,
                 "资本回报率": 0,
                 "经营活动产生的资金/债务": 0,
                 "债务/息税摊折前利润": 0,
                 "自由运营现金流/债务": 0,
                 "息税摊折前利润 / 利息支出": 0
                }

### 计算

#### 2022

In [64]:
营业利润_2022 = profit_sheet_2022.iloc[(row_to_num(7)), col_to_num("G")]
财务费用_2022 = profit_sheet_2022.iloc[(row_to_num(33)), col_to_num("C")]
折旧费_2022 = back_data.iloc[(row_to_num(2)), col_to_num("C")]
公允价值变动_2022 = profit_sheet_2022.iloc[(row_to_num(44)), col_to_num("C")]
投资收益_2022 = profit_sheet_2022.iloc[(row_to_num(39)), col_to_num("C")]
取得投资收益收到的现金_2022 = cash_sheet_2022.iloc[(row_to_num(6)), col_to_num("H")]
政府补助_2022 = profit_sheet_2022.iloc[(row_to_num(9)), col_to_num("G")]
经营租赁费用调整_2022 = back_data.iloc[(row_to_num(3)), col_to_num("C")]
资本化开发成本_2022 = back_data.iloc[(row_to_num(4)), col_to_num("C")]
勘探费用_2022 = back_data.iloc[(row_to_num(5)), col_to_num("C")]
利息收入_2022 = profit_sheet_2022.iloc[(row_to_num(35)), col_to_num("C")]
对联营企业和合营企业的投资收益_2022 = profit_sheet_2022.iloc[(row_to_num(40)), col_to_num("C")]
经营租赁的利息调整_2022 = back_data.iloc[(row_to_num(6)), col_to_num("C")]
经营活动产生的现金流量净额_2022 = cash_sheet_2022.iloc[(row_to_num(32)), col_to_num("D")]
购建固定资产无形资产和其他长期资产支付的现金_2022 = cash_sheet_2022.iloc[(row_to_num(11)), col_to_num("H")]
收到其他与投资活动有关的现金附注利息部分_2022 = back_data.iloc[(row_to_num(7)), col_to_num("C")]
分配股利利润或偿付利息支付的现金_2022 = cash_sheet_2022.iloc[(row_to_num(25)), col_to_num("H")]
对所有者或股东的分配_2022 = back_data.iloc[(row_to_num(8)), col_to_num("C")]
经营租赁折旧调整_2022 = back_data.iloc[(row_to_num(9)), col_to_num("C")]
所得税费用_2022 = profit_sheet_2022.iloc[(row_to_num(12)), col_to_num("G")]
资本化利息_2022 = back_data.iloc[(row_to_num(10)), col_to_num("C")]
短期借款_2022 = bal_sheet_2022_con.iloc[(row_to_num(6)), col_to_num("D")]
应付利息_2022 = back_data.iloc[(row_to_num(11)), col_to_num("C")]
一年内到期的长期借款_2022 = back_data.iloc[(row_to_num(12)), col_to_num("C")]
一年内到期的应付债券_2022 = back_data.iloc[(row_to_num(13)), col_to_num("C")]
其它流动负债短期应付债券_2022 = back_data.iloc[(row_to_num(14)), col_to_num("C")]
一年内应付融资租赁款_2022 = back_data.iloc[(row_to_num(15)), col_to_num("C")]
长期借款_2022 = bal_sheet_2022_con.iloc[(row_to_num(37)), col_to_num("D")]
应付债券_2022 = bal_sheet_2022_con.iloc[(row_to_num(38)), col_to_num("D")]
长期应付融资租赁款_2022 = back_data.iloc[(row_to_num(16)), col_to_num("C")]
重大合同及履行状况担保情况_2022 = back_data.iloc[(row_to_num(17)), col_to_num("C")]
货币资金_2022 = bal_sheet_2022.iloc[(row_to_num(6)), col_to_num("D")]
以公允价值计量且其变动计入当期损益的金融资产_2022 = bal_sheet_2022.iloc[(row_to_num(10)), col_to_num("D")]
其他货币资金_2022 = back_data.iloc[(row_to_num(21)), col_to_num("C")]
卖出回购金融资产款_2022 = back_data.iloc[(row_to_num(18)), col_to_num("C")]
特定行业或公司现金盈余不做调整扣除的部分加回_2022 = back_data.iloc[(row_to_num(19)), col_to_num("C")]
经营租赁调整_2022 = back_data.iloc[(row_to_num(20)), col_to_num("C")]
永续债_2022 = bal_sheet_2022_con.iloc[(row_to_num(40)), col_to_num("D")]

In [52]:
print(cash_sheet_2022.iloc[(row_to_num(32)), col_to_num("D")])
print(购建固定资产无形资产和其他长期资产支付的现金_2022)

67874.8
34323647.16


In [66]:
raw_data_2022["EBITDA"] = EBITDA(营业利润_2022,
                                 财务费用_2022,
                                 折旧费_2022,
                                 公允价值变动_2022,
                                 投资收益_2022,
                                 取得投资收益收到的现金_2022,
                                 政府补助_2022,
                                 经营租赁费用调整_2022,
                                 资本化开发成本_2022,
                                 勘探费用_2022)

raw_data_2022["EBIT"] = EBIT(营业利润_2022,
                            财务费用_2022,
                            利息收入_2022,
                            公允价值变动_2022,
                            投资收益_2022,
                            对联营企业和合营企业的投资收益_2022,
                            政府补助_2022,
                            经营租赁的利息调整_2022)

raw_data_2022["自由运营现金流(FOCF)"] = FOCF(经营活动产生的现金流量净额_2022,
                                         购建固定资产无形资产和其他长期资产支付的现金_2022,
                                         取得投资收益收到的现金_2022,
                                         收到其他与投资活动有关的现金附注利息部分_2022,
                                         分配股利利润或偿付利息支付的现金_2022,
                                         对所有者或股东的分配_2022,
                                         经营租赁折旧调整_2022,
                                         资本化开发成本_2022)

raw_data_2022["经营活动产生的现金(FFO)"] = FFO(raw_data_2022["EBITDA"],
                                           财务费用_2022,
                                           利息收入_2022,
                                           所得税费用_2022,
                                           经营租赁费用调整_2022,
                                           经营租赁折旧调整_2022,
                                           资本化利息_2022)

raw_data_2022["总负债"] = Total_debt(短期借款_2022,
                                  应付利息_2022,
                                  一年内到期的长期借款_2022,
                                  一年内到期的应付债券_2022,
                                  其它流动负债短期应付债券_2022,
                                  一年内应付融资租赁款_2022,
                                  长期借款_2022,
                                  应付债券_2022,
                                  长期应付融资租赁款_2022,
                                  重大合同及履行状况担保情况_2022,
                                  货币资金_2022,
                                  以公允价值计量且其变动计入当期损益的金融资产_2022,
                                  其他货币资金_2022,
                                  卖出回购金融资产款_2022,
                                  特定行业或公司现金盈余不做调整扣除的部分加回_2022,
                                  经营租赁调整_2022,
                                  永续债_2022)



In [70]:
print("EBITDA:", raw_data_2022["EBITDA"])
print("EBIT:", raw_data_2022["EBIT"])
print("FOCF (自由运营现金流):", raw_data_2022["自由运营现金流(FOCF)"])
print("FFO (经营活动产生的现金):", raw_data_2022["经营活动产生的现金(FFO)"])
print("Total Debt (总负债):", raw_data_2022["总负债"])


EBITDA: 163125.15
EBIT: 163125.15
FOCF (自由运营现金流): -38220001.56
FFO (经营活动产生的现金): 163125.15
Total Debt (总负债): 16581543.907499999
